#### Load Model
Load the generation model and entailment judge model.

In [ ]:

import torch
import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from seper.calculate import gen_answers_batch, calculate_uncertainty_soft_batch, create_collate_fn
from seper.models.huggingface_models import HuggingfaceModel
from seper.uncertainty_measures.semantic_entropy import EntailmentDeberta
from seper.calculate import process_item_for_seper
export http_proxy=http://192.168.32.28:18000 && export https_proxy=http://192.168.32.28:18000

model_path = 'Qwen/Qwen3-8B'
num_generations = 10 # 10 is good for most cases
sub_batch_size = 10
temperature = 1.0
max_new_tokens = 128
max_context_words = 4096
computation_chunk_size = 8 # adjust to balance speeds and gpu memory cost
prompt_type = 'default'
device = 'cuda:1'

# Build generator
generator = HuggingfaceModel(
    model_path,
    stop_sequences='default',
    max_new_tokens=max_new_tokens,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2",
    device=device,
)
generator.model.eval()

# Build entailment model
entailment_model = EntailmentDeberta(device=device)
entailment_model.model.eval()

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen3-8B/resolve/main/tokenizer_config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff646643640>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: 5b0c80f4-8926-410e-bcb6-27f18ff5a907)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-8B/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen3-8B/resolve/main/tokenizer_config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff6466431f0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: b26f2a3c-fccd-4977-b8b9-2a65f8fed8db)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-8B/resolve/main/tokenizer_config.json
Retrying i

KeyboardInterrupt: 

#### Step 1: Sampling
Generate for `num_generations` times and collect the results and probabilities.

In [ ]:
example={
    'question' : "In what city was the band behind the album Love Bites formed?",
    'context'  : """Doc 1(Title: "Love Bites (band)") Love Bites (band) Love Bites is an English girl band that formed in 2004 and disbanded in 2007, but reformed again in 2011. The band, named after a Def Leppard song, formed while the members were still in school. They started out as a three piece group consisting of Danielle Graham, Aimee Haddon and Hannah Haddon with Nicki Wood later joining the band. They did some touring and recording. In 2006 Wood left the band and was replaced by Beka Pritchard. They broke up the next year. The band released their debut single, "You Broke My Heart", on October 2005.
Doc 2(Title: "Lovebites (band)") Lovebites (band) Lovebites (stylized as LOVEBITES) is a Japanese all-female heavy metal band, formed in 2016 by former Destrose members Miho and Haruna. They derived their name from the song "Love Bites (So Do I)" by Halestorm. Lovebites was formed in Tokyo in 2016 by bassist Miho and drummer Haruna. The two met while members of Destrose, another all-female metal band that disbanded in 2015. After recruiting guitarist Midori and support guitarist and keyboardist Miyako (then known as Mi-Ya), the four chose vocalist Asami based on a demo she made. Midori was in the band Gekijo Metalicche, Miyako was a
Doc 3(Title: "Love Bites (album)") their particular brand of pop and their disillusionment with its restrictions. Producer Martin Rushent clarifies the elements of the sound even further, and Shelley's songwriting continues to improve"". In a retrospective review, BBC Music described it as ""an essential purchase for anyone remotely interested in punk's history."" AllMusic wrote: ""More musically accomplished, more obsessively self-questioning, and with equally energetic yet sometimes gloomy performances, ""Love Bites"" finds the Buzzcocks coming into their own."" Love Bites (album) Love Bites is the second studio album by English punk rock band Buzzcocks. It was released on 22 September 1978, through United Artists, on which album is the band behind the album Love Bites formed?
Doc 1(Title: Buzzcocks) Buzzcocks Buzzcocks are an English punk rock band, formed in Bolton, England, in 1976 by singer-songwriter-guitarist Pete Shelley and singer-songwriter Howard Devoto. They are regarded as a seminal influence on the Manchester music scene, the independent record label movement, punk rock, power pop, and pop punk. They achieved commercial success with singles that fused pop craftsmanship with rapid-fire punk energy. These singles were collected on ""Singles Going Steady"", described by critic Ned Raggett as a ""punk masterpiece"". Devoto and Shelley chose the name ""Buzzcocks"" after reading the headline, ""It's the Buzz, Cock!"", in a review of the TV series ""Rock
Doc 2(Title: Buzzcocks) Buzzcocks Buzzcocks are an English punk rock band, formed in Bolton, England, in 1976 by singer-songwriter-guitarist Pete Shelley and singer-songwriter Howard Devoto. They are regarded as a seminal influence on the Manchester music scene, the independent record label movement, punk rock, power pop, and pop punk. They achieved commercial success with singles that fused pop craftsmanship with rapid-fire punk energy. These singles were collected on ""Singles Going Steady"", described by critic Ned Raggett as a ""punk masterpiece"". Devoto and Shelley chose the name ""Buzzcocks"" after reading the headline, ""It's the Buzz, Cock!"", in a review of the TV series ""Rock
Doc 3(Title: Buzzcocks) the Institute, responded to the notice. Trafford had previously been involved in electronic music, while McNeish had played rock. By late 1975, Trafford and McNeish had recruited a drummer and formed, in effect, an embryonic version of Buzzcocks. The band formed, officially, in February 1976; McNeish assumed the stage name Pete Shelley and Trafford named himself Howard Devoto. They performed live for the first time on 1 April 1976 at their college. Garth Davies played bass guitar and Mick Singleton played drums. Singleton also played in local band Black Cat Bone. After reading an ""NME"" review of the Sex Pistols'""",
    'answers' : ['Bolton'] # this is the provided ground-truth answers
}
result = gen_answers_batch(example, 
                           generator, 
                           temperature, 
                           num_generations, 
                           sub_batch_size, 
                           max_new_tokens, 
                           prompt_type, 
                           device,
                           max_context_words)

# Baseline example with empty context
example_baseline = example.copy()
example_baseline['context'] = ''

result_baseline = gen_answers_batch(
    example_baseline,
    generator,
    temperature,
    num_generations,
    sub_batch_size,
    max_new_tokens,
    prompt_type,
    device,
    max_context_words
)

In [4]:
result

{'question': 'Is Elon Musk older than Sam Altman?',
 'context': 'Elon Musk was born on June 28, 1971. Sam Altman was born on April 22, 1985.',
 'answers': ['Yes'],
 'responses': [('Yes. Elon Musk was born 9 years before Sam Altman.',
   [-0.011683372780680656,
    -0.22827866673469543,
    -1.8856996297836304,
    -1.0728830375228426e-06,
    -1.9192511899746023e-05,
    -3.814689989667386e-06,
    -1.063927412033081,
    -1.3947389561508317e-05,
    -1.4852174520492554,
    -0.06590946763753891,
    -0.000164018536452204,
    -0.011408685706555843,
    -0.00011860620725201443,
    -4.851700214203447e-05,
    -3.814689989667386e-06,
    -0.0003120412293355912]),
  ('Yes.', [-0.011683372780680656, -0.22827866673469543]),
  ('Yes. Elon Musk was born in 1971, while Sam Altman was born in 1985, making Elon Musk older than Sam Altman.',
   [-0.011683372780680656,
    -0.22827866673469543,
    -1.8856996297836304,
    -1.0728830375228426e-06,
    -1.9192511899746023e-05,
    -3.8146899896673

#### Step 2: Calculate ΔSePer
Aggregate semantic-equivalent answers and compute belief shift on gt answer.

In [5]:
keys = ['question', 'response_text', 'answers', 'likelihood', 'context_label', 'log_liks_agg', 'context']
seper_collate_fn = create_collate_fn(keys)

# calculate seper
with torch.no_grad():
    # Convert for SEPER
    r = process_item_for_seper(result)
    rb = process_item_for_seper(result_baseline)
    seper_input = seper_collate_fn([r, rb])
    seper, seper_baseline = calculate_uncertainty_soft_batch(
        seper_input, entailment_model, computation_chunk_size
    )
    d_seper = seper - seper_baseline

### Extented Results: Fine-grained Retrieval Utility Evaluation

In [6]:
example_baseline={
    'question': 'Is Elon Musk older than Sam Altman?',
    'context': '',
    'answers': ['Yes'], # this is the provided ground-truth answers
}

example_piece1 = example_baseline.copy()
example_piece1['context'] = 'Elon Musk was born on June 28, 1971.'
example_piece2 = example_baseline.copy()
example_piece2['context'] = 'Sam Altman was born on April 22, 1985.'

result_piece1 = gen_answers_batch(example_piece1, generator, temperature, num_generations, sub_batch_size, max_new_tokens, prompt_type, device, max_context_words)
result_piece2 = gen_answers_batch(example_piece2, generator, temperature, num_generations, sub_batch_size, max_new_tokens, prompt_type, device, max_context_words)


In [7]:
# calculate seper
with torch.no_grad():
    # Convert for SEPER
    r_1 = process_item_for_seper(result_piece1)
    r_2 = process_item_for_seper(result_piece2)
    seper_input = seper_collate_fn([r_1, r_2])
    seper_piece1, seper_piece2 = calculate_uncertainty_soft_batch(
        seper_input, entailment_model, computation_chunk_size
    )
    d_seper_piece1 = seper_piece1 - seper_baseline
    d_seper_piece2 = seper_piece2 - seper_baseline

print(
    f'(Baseline) SePer without retrieved information: {seper_baseline}\n', 
    f'SePer with two pieces of information: {seper}, ΔSePer: {d_seper}\n', 
    f'SePer with one piece of information: {seper_piece1}, ΔSePer: {d_seper_piece1}\n', 
    f'SePer with another piece of information: {seper_piece2}, ΔSePer: {d_seper_piece2}')

(Baseline) SePer without retrieved information: 0.5311136874747243
 SePer with two pieces of information: 0.9920210827879619, ΔSePer: 0.46090739531323754
 SePer with one piece of information: 0.6686031010404139, ΔSePer: 0.13748941356568956
 SePer with another piece of information: 0.8733925752846344, ΔSePer: 0.3422788878099101
